In [1]:
from ansys.dpf.core import operators as ops
import ansys.dpf.core as dpf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import plotly.graph_objects as go
from pathlib import Path
import re 
from datetime import datetime
import pywt
from scipy.integrate import cumulative_trapezoid

# Set the correct path to the library of simulation results
base_directory = r"D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper"
figure_dir = os.path.join(base_directory, "figures")
os.makedirs(figure_dir, exist_ok=True)

In [2]:
def find_binout_files(base_dir):
    """
    Recursively find all 'binout' files (case-insensitive) within the given base directory.
    Returns a list of tuples (folder_name, file_path).
    """
    binout_files = []

    # Search for all files in the directory and filter those named "binout" (case-insensitive)
    for file_path in Path(base_dir).rglob("*"):
        if file_path.is_file() and file_path.name.lower() == "binout":
            folder_name = file_path.parent.name  # Extract folder name
            binout_files.append((folder_name, str(file_path)))

    return binout_files

def find_computed_time(base_dir):
    """
    Recursively finds all 'messag' files (case-insensitive) within the given base directory.
    Extracts the elapsed time or calculates it from start and end times.
    Returns a list of tuples (folder_name, elapsed_time, start_time, end_time).
    """
    msg_files = []
    time_data = []

    # Search for all files in the directory and filter those named "messag" (case-insensitive)
    for file_path in Path(base_dir).rglob("messag"):
        if file_path.is_file():
            folder_name = file_path.parent.name  # Extract folder name
            upper_folder = file_path.parent.parent.name # Extract Variable set
            elapsed_time = None
            start_time = None
            end_time = None

            with open(file_path, "r", encoding="utf-8") as file:
                lines = file.readlines()

            # Regex patterns
            elapsed_time_pattern = re.compile(r"Elapsed time\s+(\d+) seconds")
            start_time_pattern = re.compile(r"Start time\s+([\d/]+) ([\d:]+)")
            end_time_pattern = re.compile(r"End time\s+([\d/]+) ([\d:]+)")

            for line in lines:
                # Extract elapsed time
                elapsed_match = elapsed_time_pattern.search(line)
                if elapsed_match:
                    elapsed_time = int(elapsed_match.group(1))

                # Extract start time
                start_match = start_time_pattern.search(line)
                if start_match:
                    start_time = f"{start_match.group(1)} {start_match.group(2)}"

                # Extract end time
                end_match = end_time_pattern.search(line)
                if end_match:
                    end_time = f"{end_match.group(1)} {end_match.group(2)}"

            # If elapsed time isn't found, calculate it from start and end time
            if elapsed_time is None and start_time and end_time:
                try:
                    start_dt = datetime.strptime(start_time, "%m/%d/%Y %H:%M:%S")
                    end_dt = datetime.strptime(end_time, "%m/%d/%Y %H:%M:%S")
                    elapsed_time = int((end_dt - start_dt).total_seconds())
                except ValueError:
                    pass  # Skip calculation if datetime parsing fails

            time_data.append((upper_folder, folder_name, elapsed_time, start_time, end_time))

    return time_data

def find_number_elements(base_dir):
    """
    Recursively finds all 'd3plot' files in the given base directory and extracts the total number of elements.
    Returns a list of tuples (folder_name, num_elements).
    """
    element_counts = []

    # Search for all 'd3plot' files in the directory
    for d3plot_path in Path(base_dir).rglob("d3plot"):
        if d3plot_path.is_file():
            folder_name = d3plot_path.parent.name  # Extract folder name
            upper_folder = d3plot_path.parent.parent.name # Extract varibale set

            # Initialize DPF data source
            ds_full = dpf.DataSources()
            ds_full.set_result_file_path(str(d3plot_path), "d3plot")

            # Load the model
            model_full = dpf.Model(ds_full)

            # Get the mesh
            mesh = model_full.metadata.meshed_region

            # Extract total number of elements
            element_ids = mesh.elements.scoping.ids
            num_elements = len(element_ids)

            element_counts.append((upper_folder, folder_name, num_elements))

    return element_counts

# Define mrybm color scale
mrybm_colors = [
            "rgb(0, 0, 255)",   # Blue
            "rgb(255, 0, 0)",   # Red
            "rgb(0, 128, 0)",   # Green
            "rgb(255, 165, 0)", # Orange                                   
            "rgb(75, 0, 130)",  # Indigo
            "rgb(238, 130, 238)"# Violet
        ]


In [55]:
# Define the base directory
shaper_thicknesses = ["025in", "020in", "015in"]
striker_velocities = ["15ms", "20ms", "25ms"]

for shaper_t in shaper_thicknesses:
    for striker_v in striker_velocities:

        folder_directory = os.path.join(base_directory, shaper_t, striker_v)
        
        # Get BINOUT files with associated folder names
        binout_data = find_binout_files(folder_directory)
        print(f"Number of BINOUT files found: {len(binout_data)}")    
        
        strain_raw_fig = go.Figure()
        
        strain_raw_fig.update_layout(width=1100, height=600, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                                     title=dict(text="Strain Signal vs Time", x=0.5, y=0.95, xanchor="center",
                                                font=dict(size=20, color="black", family="Arial")),
                                     xaxis=dict(title=dict(text="Time (ms)", font=dict(family="Arial", size=16, color="black")),
                                                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                                                gridcolor="lightgrey", gridwidth=1),
                                     yaxis=dict(title=dict(text="Strain", font=dict(family="Arial", size=16, color="black")),
                                                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                                                gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                                     legend=dict(title="Shaper Radius", x=1.0, y=1.0, font=dict(size=12, color="black", family="Arial"),
                                                 bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))
        
        strain_raw_fig.add_annotation(
            text=f"Striker Velocity = {striker_v.split('ms')[0]} m/s <br>Shaper Thickness = 0.{shaper_t.split('in')[0]} in",
            xref="paper", yref="paper", x=0, y=1, showarrow=False,
            font=dict(size=14, color="black", family="Arial"), align="left", bgcolor="white", bordercolor="black", borderwidth=2)
            
        for idx, data in enumerate(binout_data):
            
            # Create a DataSources object and a Model
            legend, file_path = data
            ds = dpf.DataSources(file_path)
            model = dpf.Model(ds)
        
            legend = ["3/16 in", "1/8 in", "3/32 in"]
            
            #available = model.metadata.result_info.available_results
            #print("Available results in binout:", available)
            
            total_strain_result = model.results.total_strain()
            strain_fc = total_strain_result.outputs.fields_container()
            
            strain_incident = strain_fc[0].data # Refers to the x-strain at the incident Strain Gauge
            strain_transmitted = strain_fc[6].data # Refers to the x-strain at the transmitted Strain Gauge
            
            time_freq_support = model.metadata.time_freq_support
            time_values = time_freq_support.time_frequencies.data
        
            color = mrybm_colors[idx % len(mrybm_colors)]
            
            strain_raw_fig.add_trace(go.Scatter(x=time_values, y=strain_incident, mode='lines',
                                                name=f"{legend[idx]}", line=dict(color=color, width=2)))
            strain_raw_fig.add_trace(go.Scatter(x=time_values, y=strain_transmitted, mode='lines',
                                               name=f"{legend[idx]}", line=dict(color=color, width=2)))
        
        # Save the interactive plot as an HTML file
        output_filename = f"Radii_Study_{shaper_t}_{striker_v}.html"
        study_figure_dir = os.path.join(figure_dir, "Radii")
        os.makedirs(study_figure_dir, exist_ok=True)
        strain_raw_fig.write_html(os.path.join(study_figure_dir, output_filename))



Number of BINOUT files found: 3
Number of BINOUT files found: 3
Number of BINOUT files found: 3
Number of BINOUT files found: 3
Number of BINOUT files found: 3
Number of BINOUT files found: 3
Number of BINOUT files found: 3
Number of BINOUT files found: 3
Number of BINOUT files found: 3


In [46]:
# Define parameters
shaper_thicknesses = ["025in", "020in", "015in"]
striker_velocities = ["15ms", "20ms", "25ms"]
shaper_radii = ["_050", "_033", "_025"]  # Corresponding to 3/16 in, 1/8 in, 3/32 in

# Define mapping for legend names
radius_labels = {"_050": "3/16 in", "_033": "1/8 in", "_025": "3/32 in"}

# Function to find BINOUT files for a specific thickness and radius across all velocities
def find_binout_files_for_radius(base_dir, thickness, radius):
    """
    Find 'binout' files for a given shaper thickness and radius across all velocities.
    Returns a dictionary where keys are velocities, and values are (folder_name, file_path) tuples.
    """
    binout_data = {}
    for velocity in striker_velocities:
        velocity_path = os.path.join(base_dir, thickness, velocity, radius)
        print(velocity_path)
        binout_files = []

        # Use the updated search function
        for file_path in Path(velocity_path).rglob("*"):
            if file_path.is_file() and file_path.name.lower() == "binout":
                folder_name = file_path.parent.name
                binout_files.append((folder_name, str(file_path)))

        if binout_files:
            binout_data[velocity] = binout_files  # Store per velocity

    return binout_data

# Loop over thicknesses and shaper radii
for shaper_t in shaper_thicknesses:
    for shaper_r in shaper_radii:
        shaper_r_folder = f"shaper_{shaper_t.split('in')[0]}{shaper_r}"
        # Find binout files for the given thickness and radius across all velocities
        binout_data = find_binout_files_for_radius(base_directory, shaper_t, shaper_r_folder)

        # Skip if no data is found for this combination
        if not binout_data:
            print(f"No binout data found")
            continue

        # Create a new figure
        strain_raw_fig = go.Figure()

        # Set layout
        strain_raw_fig.update_layout(
            width=1100, height=600, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
            title=dict(
                text=f"Strain Signal vs Time for {radius_labels[shaper_r]} Radius, {shaper_t} Thickness",
                x=0.5, y=0.95, xanchor="center",
                font=dict(size=20, color="black", family="Arial")
            ),
            xaxis=dict(
                title=dict(text="Time (ms)", font=dict(family="Arial", size=16, color="black")),
                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                gridcolor="lightgrey", gridwidth=1
            ),
            yaxis=dict(
                title=dict(text="Strain", font=dict(family="Arial", size=16, color="black")),
                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"
            ),
            legend=dict(
                title="Striker Velocity", x=1.0, y=1.0,
                font=dict(size=12, color="black", family="Arial"),
                bgcolor="#FFFFFF", bordercolor="black", borderwidth=2
            )
        )

        # Add a text box annotation
        strain_raw_fig.add_annotation(
            text=f"Shaper Radius = {radius_labels[shaper_r]}<br>Shaper Thickness = 0.{shaper_t.split('in')[0]} in",
            xref="paper", yref="paper", x=0, y=1, showarrow=False,
            font=dict(size=14, color="black", family="Arial"), align="left",
            bgcolor="white", bordercolor="black", borderwidth=2
        )

        # Loop through available velocity data
        for idx, velocity in enumerate(binout_data.keys()):
            for folder_name, file_path in binout_data[velocity]:
                
                # Create a DataSources object and a Model
                ds = dpf.DataSources(file_path)
                model = dpf.Model(ds)

                # Extract strain data
                total_strain_result = model.results.total_strain()
                strain_fc = total_strain_result.outputs.fields_container()

                strain_incident = strain_fc[0].data  # Incident strain
                strain_transmitted = strain_fc[6].data  # Transmitted strain

                time_freq_support = model.metadata.time_freq_support
                time_values = time_freq_support.time_frequencies.data

                # Assign color for each velocity
                color = mrybm_colors[idx % len(mrybm_colors)]

                # Add traces for incident and transmitted strains
                strain_raw_fig.add_trace(go.Scatter(
                    x=time_values, y=strain_incident, mode='lines',
                    name=f"{velocity.split('ms')[0]} m/s - Incident", line=dict(color=color, width=2)
                ))
                strain_raw_fig.add_trace(go.Scatter(
                    x=time_values, y=strain_transmitted, mode='lines',
                    name=f"{velocity.split('ms')[0]} m/s - Transmitted", line=dict(color=color, width=2, dash="dash")
                ))

        # Save the interactive plot as an HTML file
        output_filename = f"Velocity_Study_{shaper_t}{shaper_r}.html"
        study_figure_dir = os.path.join(figure_dir, "velocity")
        os.makedirs(study_figure_dir, exist_ok=True)
        strain_raw_fig.write_html(os.path.join(study_figure_dir, output_filename))
        print(f"Completed: {shaper_t} thickness, {radius_labels[shaper_r]} radius")


D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_050
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\20ms\shaper_025_050
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\25ms\shaper_025_050
Completed: 025in thickness, 3/16 in radius
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_033
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\20ms\shaper_025_033
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\25ms\shaper_025_033
Completed: 025in thickness, 1/8 in radius
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_025
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\20ms\shaper_025_025
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\25ms\shaper_025_025
Completed: 025in thickness, 3/32 in radius
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\020in\15ms\shaper_020_050
D:\DynaMat_UTEP\SHPB_P

In [53]:
# Define parameters
shaper_thicknesses = ["025in", "020in", "015in"]  # Now thickness is the variable
striker_velocities = ["15ms", "20ms", "25ms"]  # Keep velocity fixed per plot
shaper_radii = ["_050", "_033", "_025"]  # Keep radius fixed per plot

# Define mapping for legend names
radius_labels = {
    "_050": "3/16 in",
    "_033": "1/8 in",
    "_025": "3/32 in"
}

# Function to find BINOUT files for a specific velocity and radius across all thicknesses
def find_binout_files_for_thickness(base_dir, velocity, radius):
    """
    Find 'binout' files for a given striker velocity and radius across all thicknesses.
    Returns a dictionary where keys are thicknesses, and values are (folder_name, file_path) tuples.
    """
    binout_data = {}
    for thickness in shaper_thicknesses:
        radius_folder = f"shaper_{thickness.split('in')[0]}{radius}"
        thickness_path = os.path.join(base_dir, thickness, velocity, radius_folder)
        print(f"Searching in: {thickness_path}")
        binout_files = []

        # Search for BINOUT files
        for file_path in Path(thickness_path).rglob("*"):
            if file_path.is_file() and file_path.name.lower() == "binout":
                folder_name = file_path.parent.name
                binout_files.append((folder_name, str(file_path)))

        if binout_files:
            binout_data[thickness] = binout_files  # Store per thickness

    return binout_data

# Loop over velocities and shaper radii (keeping them constant per plot)
for striker_v in striker_velocities:
    for shaper_r in shaper_radii:
        
        # Find binout files for the given velocity and radius across all thicknesses
        binout_data = find_binout_files_for_thickness(base_directory, striker_v, shaper_r)

        # Skip if no data is found for this combination
        if not binout_data:
            print(f"No binout data found for {striker_v} velocity, {radius_labels[shaper_r]} radius")
            continue

        # Create a new figure
        strain_raw_fig = go.Figure()

        # Set layout
        strain_raw_fig.update_layout(
            width=1100, height=600, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
            title=dict(
                text=f"Strain Signal vs Time for {radius_labels[shaper_r]} Radius, {striker_v} Velocity",
                x=0.5, y=0.95, xanchor="center",
                font=dict(size=20, color="black", family="Arial")
            ),
            xaxis=dict(
                title=dict(text="Time (ms)", font=dict(family="Arial", size=16, color="black")),
                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                gridcolor="lightgrey", gridwidth=1
            ),
            yaxis=dict(
                title=dict(text="Strain", font=dict(family="Arial", size=16, color="black")),
                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"
            ),
            legend=dict(
                title="Shaper Thickness", x=1.0, y=1.0,
                font=dict(size=12, color="black", family="Arial"),
                bgcolor="#FFFFFF", bordercolor="black", borderwidth=2
            )
        )

        # Add a text box annotation
        strain_raw_fig.add_annotation(
            text=f"Shaper Radius = {radius_labels[shaper_r]}<br>Striker Velocity = {striker_v.split('ms')[0]} m/s",
            xref="paper", yref="paper", x=0, y=1, showarrow=False,
            font=dict(size=14, color="black", family="Arial"), align="left",
            bgcolor="white", bordercolor="black", borderwidth=2
        )

        # Loop through available thickness data
        for idx, thickness in enumerate(binout_data.keys()):
            for folder_name, file_path in binout_data[thickness]:
                
                # Create a DataSources object and a Model
                ds = dpf.DataSources(file_path)
                model = dpf.Model(ds)

                # Extract strain data
                total_strain_result = model.results.total_strain()
                strain_fc = total_strain_result.outputs.fields_container()

                strain_incident = strain_fc[0].data  # Incident strain
                strain_transmitted = strain_fc[6].data  # Transmitted strain

                time_freq_support = model.metadata.time_freq_support
                time_values = time_freq_support.time_frequencies.data

                # Assign color for each thickness
                color = mrybm_colors[idx % len(mrybm_colors)]

                # Add traces for incident and transmitted strains
                strain_raw_fig.add_trace(go.Scatter(
                    x=time_values, y=strain_incident, mode='lines',
                    name=f"{thickness} - Incident", line=dict(color=color, width=2)
                ))
                strain_raw_fig.add_trace(go.Scatter(
                    x=time_values, y=strain_transmitted, mode='lines',
                    name=f"{thickness} - Transmitted", line=dict(color=color, width=2, dash="dash")
                ))

        # Save the interactive plot as an HTML file
        output_filename = f"Thickness_Study_{striker_v}_{shaper_r}.html"
        study_figure_dir = os.path.join(figure_dir, "thickness")
        os.makedirs(study_figure_dir, exist_ok=True)
        strain_raw_fig.write_html(os.path.join(study_figure_dir, output_filename))
        print(f"Completed: {striker_v} velocity, {radius_labels[shaper_r]} radius")


Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_050
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\020in\15ms\shaper_020_050
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\15ms\shaper_015_050
Completed: 15ms velocity, 3/16 in radius
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_033
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\020in\15ms\shaper_020_033
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\15ms\shaper_015_033
Completed: 15ms velocity, 1/8 in radius
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_025
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\020in\15ms\shaper_020_025
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\15ms\shaper_015_025
Completed: 15ms velocity, 3/

In [34]:
def find_pulse_edges(edges, signal, pulse_window):
    """
    Identifies and classifies pulses using derivative-based edge detection and validates pulse windows.

    Parameters:
    edges (array-like): List of detected edge indices.
    signal (array-like): The reference signal (e.g., SHPB strain gauge data).
    pulse_window (float): Expected pulse duration (in index units).

    Returns:
    tuple: (List of valid pulse start indices, Dictionary of all pulse properties)
    """
    rising_edges_start_idx = []
    falling_edges_start_idx = [] 
    window_start_approx = []
    window_end_approx = []
    
    edge_results = {
        "start": [],
        "end": [],
        "n_points": [],
        "direction": []
    }

    if len(edges) == 0:
        return window_start_approx, edge_results  # Return empty result if no edges detected

    # Compute first derivative of the signal
    derivative = np.gradient(signal)

    # Sort edges to ensure order
    edges = np.sort(edges)

    # Group consecutive edges
    grouped_edges = []
    current_group = [edges[0]]

    for i in range(1, len(edges)):
        if edges[i] - edges[i - 1] <= 2500:  # Allowing small gaps (adjustable)
            current_group.append(edges[i])
        else:
            grouped_edges.append(current_group)
            current_group = [edges[i]]
    
    grouped_edges.append(current_group)  # Add last group

    # Process grouped edges to extract pulse properties
    for group in grouped_edges:
        start_idx = group[0]
        end_idx = group[-1]
        n_points = len(group)

        # Compute average derivative over the pulse range
        avg_slope = np.mean(derivative[start_idx:end_idx])

        # Direction is determined by the sign of the average derivative
        if avg_slope > 0: 
            direction = 1
            rising_edges_start_idx.append(start_idx)
        else: 
            direction = -1
            falling_edges_start_idx.append(start_idx)

        # Store results
        edge_results["start"].append(start_idx)
        edge_results["end"].append(end_idx)
        edge_results["n_points"].append(n_points)
        edge_results["direction"].append(direction)

    # **Improved Logic for Pulse Window Matching**
    for rise_idx in rising_edges_start_idx:
        # Find the closest falling edge that meets pulse window constraints
        valid_falls = [
            fall_idx for fall_idx in falling_edges_start_idx
            if pulse_window * 0.90 <= np.abs(rise_idx - fall_idx) <= pulse_window * 1.30
        ]

        if valid_falls:
            # Select the closest falling edge
            best_fall_idx = min(valid_falls, key=lambda x: abs(rise_idx - x))
            window_start_approx.append(min(rise_idx, best_fall_idx))  # Store the earlier index as the start
            window_end_approx.append(max(rise_idx, best_fall_idx))

    return window_start_approx, window_end_approx

def extract_pulse_window(signal, signal_start_approx, signal_end_approx, pulse_points, pp_extra, negative = True):
    
    search_range = np.arange(int(signal_start_approx-(pulse_points*pp_extra)),
                             int(signal_start_approx+(pulse_points*(1+pp_extra)))
                             ,1)
    
    signal_subset = np.array(signal[search_range])    
    signal_gradient = np.gradient(signal_subset)

    if negative:
        signal_start_slope = np.argmin(signal_gradient)
        
    else: 
        signal_start_slope = np.argmax(signal_gradient)
        
    signal_start_zero = np.where(np.isclose(signal_subset[:signal_start_slope], 0, atol=5e-5))[0][-1]
    signal_end_zero = np.where(np.isclose(signal_subset[signal_start_slope:], 0, atol=5e-5))[0][0]
    
    window_range = np.arange(int(signal_start_zero), int(signal_start_slope + signal_end_zero), 1)    
    extracted_signal = signal_subset[window_range]
    time_start_idx = int(signal_start_approx-(pulse_points*pp_extra) + signal_start_zero)
    
    return extracted_signal, time_start_idx

def fit_pulse_windows(pulses, threshold, negatives):
    """
    Trim a list of pulses based on a threshold so that each pulse starts at
    its own threshold crossing and is then trimmed to a common length.
    
    Parameters:
      pulses    : list or array of 1D arrays
                  Each element is a pulse (time-series data).
      threshold : float
                  The threshold value used for determining the pulse start.
      negative  : bool, default True
                  If True, the pulse start is defined as the first index 
                  where the pulse value is less than the threshold.
                  If False, the pulse start is defined as the first index 
                  where the pulse value is greater than the threshold.
    
    Returns:
      trimmed_pulses : list of 1D NumPy arrays
                       Each pulse is trimmed starting at its own threshold
                       crossing and continuing for a common length (the
                       minimum available length across pulses).
    """
    start_indices = []
    for idx, pulse in enumerate(pulses):
        # Ensure pulse is a NumPy array.
        pulse = np.array(pulse)
        
        if negatives[idx]:
            # Find indices where the pulse is below the threshold.
            indices = np.where(pulse < -threshold)[0]
        else:
            indices = np.where(pulse > threshold)[0]
        
        # If no index meets the criterion, we default to index 0.
        idx = indices[0] if len(indices) > 0 else 0
        start_indices.append(idx)
    
    # For each pulse, compute the number of points available after its start index.
    lengths_after = [len(pulse) - idx for pulse, idx in zip(pulses, start_indices)]
    common_length = min(lengths_after)
    
    # Now trim every pulse from its own start index to start index + common_length.
    trimmed_pulses = [np.array(pulse)[idx: idx + common_length]
                      for pulse, idx in zip(pulses, start_indices)]
    
    return trimmed_pulses

In [35]:
def compute_true_stress_strain(time_raw, incident_raw, transmitted_raw, debug=False):
    
    wavelet = 'gaus1'
    scales = np.arange(1, 300, 2)  # Multi-scale analysis 
    pulse_data_points = 9850 # from previous analysis in database

    dt = np.mean(np.diff(time_values))
    bar_wave_speed = 4953.321
    striker_length = 304.8 
    specimen_length = 6.36 
    bar_elastic_modulus = 2e5
    bar_cross_section = 71.251 
    specimen_cross_section = 31.76904
    pulse_duration = (2*striker_length) / bar_wave_speed # ms

    if debug:
        print("User Defined Variables for Compute True Stress - Strain func")
        print("---"*20)
        print("\nBar Properties")
        print(f"Striker Bar Length: {striker_length:.3f} mm")
        print(f"Bar Cross Section: {bar_cross_section:.3f} mm^2")
        print(f"Bar Elastic Modulus: {bar_elastic_modulus:.3e} MPa")
        print(f"Bar Wave Speed: {bar_wave_speed:.3f} m/s or mm/ms")
        print("---"*20)
        print("\nSpecimen Properties")
        print(f"Specimen Cross Section: {specimen_cross_section:.3f} mm^2")
        print(f"Specimen Length: {specimen_length:.3f} mm")
        print("---"*20)
        print("\nPulse Properties")
        print(f"Pulse Data Points: {pulse_data_points} pts")
        print(f"Pulse mean Delta T: {dt:.3e} ms")
        print(f"Pulse Duration: {pulse_duration} ms")
        print(f"Incident Raw Pulse Shape: {incident_raw.shape}, and type {type(incident_raw)}")
        print(f"Transmitted Raw Pulse Shape: {transmitted_raw.shape}, and type {type(transmitted_raw)}")
        print(f"Time Raw Shape: {time_raw.shape}, and type {type(time_raw)}")
        print("---"*20)
        print("\nSection 1, function variables complete...")
        print("---"*20)   
        
    # Compute CWT
    incident_coeffs, _ = pywt.cwt(incident_raw, scales, wavelet)
    transmitted_coeffs, _ = pywt.cwt(incident_raw, scales, wavelet)
    
    # Find strong edges by taking the absolute max of wavelet coefficients
    incident_edges = np.where(np.abs(incident_coeffs).max(axis=0) > np.percentile(np.abs(incident_coeffs), 97.5))[0]
    transmitted_edges = np.where(np.abs(transmitted_coeffs).max(axis=0) > np.percentile(np.abs(transmitted_coeffs), 97.5))[0]    

    if debug:         
        print("\nSection 2, Extracting Pulse Edges using PyWavelet...")
        print("---"*20) 
        plt.figure(figsize =(10,6))
        plt.plot(time_raw, incident_raw, label="Incident")
        plt.scatter(time_raw[incident_edges], incident_raw[incident_edges], marker ="o", c="lightblue", label = "Incident Edges")
        plt.plot(time_raw, transmitted_raw, label="Transmitted")
        plt.scatter(time_raw[transmitted_edges], transmitted_raw[transmitted_edges], marker = "o", c="red", label="Transmitted Edges")
        plt.title("SHPB Raw Data Edges")
        plt.legend()
        plt.xlabel("Time (ms)")
        plt.ylabel("Strain (mm/mm)")
        plt.show()

  
    incident_start_approx, incident_end_approx = find_pulse_edges(incident_edges, strain_incident,
                                                                          pulse_data_points)
    transmitted_start_approx, transmitted_end_approx = find_pulse_edges(transmitted_edges, strain_transmitted,
                                                                                    pulse_data_points)

    if debug:         
        print("\nSection 2, Extracting Pulse Edges using PyWavelet...")
        print("---"*20) 
        plt.figure(figsize =(10,6))
        plt.title("SPHB Data, Pulse Approximation Location")
        plt.plot(time_values, strain_incident, label="Incident")
        plt.scatter(time_values[incident_start_approx], strain_incident[incident_start_approx], marker ="o", c="lightblue", label = "Incident Targets")
        plt.scatter(time_values[incident_end_approx], strain_incident[incident_end_approx], marker ="o", c="lightblue")
        plt.plot(time_values, strain_transmitted, label="Transmitted")
        plt.scatter(time_values[transmitted_start_approx], strain_transmitted[transmitted_start_approx], marker = "o", c="red", label = "Incident Targets")
        plt.scatter(time_values[transmitted_end_approx], strain_transmitted[transmitted_end_approx], marker = "o", c="red")
        plt.legend()
        plt.xlabel("Time (ms)")
        plt.ylabel("Strain (mm/mm)")
        plt.show()

        
 
    incident_extracted, _ = extract_pulse_window(incident_raw, incident_start_approx[0],
                                                                     incident_end_approx[0], pulse_data_points, 2)
    transmitted_extracted, _ = extract_pulse_window(transmitted_raw, transmitted_start_approx[0],
                                                                     transmitted_end_approx[0], pulse_data_points, 2)
    reflected_extracted, _ = extract_pulse_window(incident_raw, incident_start_approx[1],
                                                                         incident_end_approx[1], pulse_data_points, 2, negative=False)
    subset_points = min(len(incident_extracted), len(reflected_extracted), len(transmitted_extracted))
                 
    incident_extracted = incident_extracted[:subset_points]
    transmitted_extracted = transmitted_extracted[:subset_points]
    reflected_extracted = reflected_extracted[:subset_points]   
    time_extracted = np.linspace(0, pulse_duration, subset_points)
        
    if debug:
        print("\nSection 3, Extracting Pulse Edges from Approximation...")
        print("---"*20) 
        plt.figure(figsize =(10,6))
        plt.title("SPHB Data, Pulse Extraction")
        plt.plot(time_extracted, incident_extracted, label="Incident")
        plt.plot(time_extracted, transmitted_extracted, label="Transmitted")
        plt.plot(time_extracted, reflected_extracted, label="Reflected")
        plt.legend()
        plt.xlabel("Time (ms)")
        plt.ylabel("Strain (mm/mm)")
        plt.show()
        
    fitted_pulses = fit_pulse_windows([incident_extracted, reflected_extracted, transmitted_extracted], 1e-4, [True,False,True])
    incident_corrected = np.array(fitted_pulses[0])
    reflected_corrected = np.array(fitted_pulses[1])
    transmitted_corrected = np.array(fitted_pulses[2])
        
    time_corrected = time_extracted[len(time_extracted) - len(incident_corrected):] 

    if debug: 
        print("\nSection 4, Fitted Extracted Pulses...")
        print("---"*20) 
        plt.figure(figsize =(10,6))
        plt.title("SPHB Data, Pulse Fitting")
        plt.plot(time_corrected, incident_corrected, label="Incident")
        plt.plot(time_corrected, transmitted_corrected, label="Transmitted")
        plt.plot(time_corrected, reflected_corrected, label="Reflected")
        plt.legend()
        plt.xlabel("Time (ms)")
        plt.ylabel("Strain (mm/mm)")
        plt.show()


    u1_corr = bar_wave_speed * (incident_corrected + reflected_corrected)
    u2_corr = bar_wave_speed * (transmitted_corrected)
    
    strain_rate_1_corr = (bar_wave_speed / specimen_length) * (incident_corrected - reflected_corrected - transmitted_corrected)
    strain_rate_2_corr = (2*bar_wave_speed*reflected_corrected) / specimen_length
    
    strain_1_corr = (bar_wave_speed / specimen_length) * cumulative_trapezoid(
                            (incident_corrected - reflected_corrected - transmitted_corrected),
                            time_corrected, initial=0)
            
    strain_2_corr = ((2*bar_wave_speed) / specimen_length) * cumulative_trapezoid(
                            reflected_corrected,
                            time_corrected, initial=0)
    
    sigma_1_corr = (bar_elastic_modulus/1000) * (bar_cross_section / specimen_cross_section) * (incident_corrected + reflected_corrected)
    sigma_2_corr = (bar_elastic_modulus/1000) * (bar_cross_section / specimen_cross_section) * (transmitted_corrected)
    
    true_strain_1_corr = np.log(1 + strain_1_corr)
    true_strain_2_corr = np.log(1 + strain_2_corr)
    true_stress_1_corr = sigma_1_corr * (1 + strain_1_corr)
    true_stress_2_corr = sigma_2_corr * (1 + strain_2_corr)
    
    if debug:
        print("\nSection 5, SHPB Analysis...")
        print("---"*20) 
        plt.figure(figsize =(10,6))
        plt.title("SPHB Data, Bar Displacement")
        plt.plot(time_corrected, -u1_corr, label="2-Wave Method")
        plt.plot(time_corrected, -u2_corr, label="1-Wave Method")
        plt.legend()
        plt.xlabel("Time (ms)")
        plt.ylabel("Velocity (mm/ms)")
        plt.show()

        plt.figure(figsize =(10,6))
        plt.title("SPHB Data, Engineering Strain Rate")
        plt.plot(time_corrected, -strain_rate_1_corr*1000, label="3-Wave Method")
        plt.plot(time_corrected, strain_rate_2_corr*1000, label="1-Wave Method")
        plt.legend()
        plt.xlabel("Time (ms)")
        plt.ylabel("Strain Rate (1/s)")
        plt.show()
        
        plt.figure(figsize =(10,6))
        plt.title("SPHB Data, Engineering Strain")
        plt.plot(time_corrected, -strain_1_corr, label="3-Wave Method")
        plt.plot(time_corrected, strain_2_corr, label="1-Wave Method")
        plt.legend()
        plt.xlabel("Time (ms)")
        plt.ylabel("Engineering Strain (mm/mm)")
        plt.show()
        
        plt.figure(figsize =(10,6))
        plt.title("SPHB Data, Engineering Stress")
        plt.plot(time_corrected, -sigma_1_corr*1000, label="2-Wave Method")
        plt.plot(time_corrected, -sigma_2_corr*1000, label="1-Wave Method")
        plt.legend()
        plt.xlabel("Time (ms)")
        plt.ylabel("Engineering Stress (MPa)")
        plt.show()

        plt.figure(figsize =(10,6))
        plt.title("SPHB Data, Engineering Stress vs Strain")        
        plt.plot(-strain_1_corr, -sigma_1_corr*1000, label="N-Wave Method")
        plt.plot(strain_2_corr, -sigma_2_corr*1000, label="1-Wave Method")
        plt.legend()
        plt.xlabel("Engineering Strain (mm/mm)")
        plt.ylabel("Engineering Stress (MPa)")
        plt.show()

        plt.figure(figsize =(10,6))
        plt.title("SPHB Data, True Stress vs Strain") 
        plt.plot(-true_strain_1_corr, -true_stress_1_corr, label="N-Wave Method")
        plt.plot(true_strain_2_corr, -true_stress_2_corr, label="1-Wave Method")
        plt.legend()
        plt.xlabel("True Strain (mm/mm)")
        plt.ylabel("True Stress (MPa)")
        plt.show()

    return true_strain_1_corr, true_strain_2_corr, true_stress_1_corr, true_stress_2_corr

In [38]:
# Define the base directory
shaper_thicknesses = ["025in", "020in", "015in"]
striker_velocities = ["15ms", "20ms", "25ms"]

for shaper_t in shaper_thicknesses[-1:]:
    for striker_v in striker_velocities[1:]:

        folder_directory = os.path.join(base_directory, shaper_t, striker_v)
        
        # Get BINOUT files with associated folder names
        binout_data = find_binout_files(folder_directory)
        print(f"Number of BINOUT files found: {len(binout_data)}")    
        
        true_stress_strain = go.Figure()
        
        true_stress_strain.update_layout(width=1100, height=600, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                                     title=dict(text="True Stress vs True Strain", x=0.5, y=0.95, xanchor="center",
                                                font=dict(size=20, color="black", family="Arial")),
                                     xaxis=dict(title=dict(text="True Strain (mm/mm)", font=dict(family="Arial", size=16, color="black")),
                                                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                                                gridcolor="lightgrey", gridwidth=1),
                                     yaxis=dict(title=dict(text="True Stress (MPa)", font=dict(family="Arial", size=16, color="black")),
                                                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                                                gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                                     legend=dict(title="Shaper Radius - Wave Method", x=1.0, y=1.0, font=dict(size=12, color="black", family="Arial"),
                                                 bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))
        
        true_stress_strain.add_annotation(
            text=f"Striker Velocity = {striker_v.split('ms')[0]} m/s <br>Shaper Thickness = 0.{shaper_t.split('in')[0]} in",
            xref="paper", yref="paper", x=0, y=1, showarrow=False,
            font=dict(size=14, color="black", family="Arial"), align="left", bgcolor="white", bordercolor="black", borderwidth=2)
            
        for idx, data in enumerate(binout_data):
            
            # Create a DataSources object and a Model
            legend, file_path = data
            print(file_path)
            ds = dpf.DataSources(file_path)
            model = dpf.Model(ds)
        
            legend = ["3/16 in", "1/8 in", "3/32 in"]
            
            #available = model.metadata.result_info.available_results
            #print("Available results in binout:", available)
            
            total_strain_result = model.results.total_strain()
            strain_fc = total_strain_result.outputs.fields_container()
            
            strain_incident = strain_fc[0].data # Refers to the x-strain at the incident Strain Gauge
            strain_transmitted = strain_fc[6].data # Refers to the x-strain at the transmitted Strain Gauge
            
            time_freq_support = model.metadata.time_freq_support
            time_values = time_freq_support.time_frequencies.data
        
            color = mrybm_colors[idx % len(mrybm_colors)]
            try: 
                true_strain_1, true_strain_2, true_stress_1, true_stress_2 = compute_true_stress_strain(time_values,
                                                                                                        strain_incident,
                                                                                                        strain_transmitted,
                                                                                                        debug=False)
                
                true_stress_strain.add_trace(go.Scatter(x=-true_strain_1, y=-true_stress_1*1000, mode='lines',
                                                    name=f"{legend[idx]} - 2-Wave Method", line=dict(color=color, width=2)))
                true_stress_strain.add_trace(go.Scatter(x=true_strain_2, y=-true_stress_2*1000, mode='lines',
                                                    name=f"{legend[idx]} - 1-Wave Method", line=dict(color=color, width=2)))
            except Exception as e:
                print(e)
        
        # Save the interactive plot as an HTML file
        output_filename = f"Radii_Study_{shaper_t}_{striker_v}.html"
        study_figure_dir = os.path.join(figure_dir, "Radii_True")
        os.makedirs(study_figure_dir, exist_ok=True)
        true_stress_strain.write_html(os.path.join(study_figure_dir, output_filename))



Number of BINOUT files found: 3
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\20ms\shaper_015_050\binout
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\20ms\shaper_015_033\binout
list index out of range
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\20ms\shaper_015_025\binout
Number of BINOUT files found: 3
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\25ms\shaper_015_050\binout
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\25ms\shaper_015_033\binout
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\25ms\shaper_015_025\binout


In [41]:
# Define parameters
shaper_thicknesses = ["025in", "020in", "015in"]
striker_velocities = ["15ms", "20ms", "25ms"]
shaper_radii = ["_050", "_033", "_025"]  # Corresponding to 3/16 in, 1/8 in, 3/32 in

# Define mapping for legend names
radius_labels = {"_050": "3/16 in", "_033": "1/8 in", "_025": "3/32 in"}

# Function to find BINOUT files for a specific thickness and radius across all velocities
def find_binout_files_for_radius(base_dir, thickness, radius):
    """
    Find 'binout' files for a given shaper thickness and radius across all velocities.
    Returns a dictionary where keys are velocities, and values are (folder_name, file_path) tuples.
    """
    binout_data = {}
    for velocity in striker_velocities:
        velocity_path = os.path.join(base_dir, thickness, velocity, radius)
        print(velocity_path)
        binout_files = []

        # Use the updated search function
        for file_path in Path(velocity_path).rglob("*"):
            if file_path.is_file() and file_path.name.lower() == "binout":
                folder_name = file_path.parent.name
                binout_files.append((folder_name, str(file_path)))

        if binout_files:
            binout_data[velocity] = binout_files  # Store per velocity

    return binout_data

# Loop over thicknesses and shaper radii
for shaper_t in shaper_thicknesses:
    for shaper_r in shaper_radii:
        shaper_r_folder = f"shaper_{shaper_t.split('in')[0]}{shaper_r}"
        # Find binout files for the given thickness and radius across all velocities
        binout_data = find_binout_files_for_radius(base_directory, shaper_t, shaper_r_folder)

        # Skip if no data is found for this combination
        if not binout_data:
            print(f"No binout data found")
            continue

        true_stress_strain = go.Figure()
        
        true_stress_strain.update_layout(width=1100, height=600, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                                     title=dict(text="True Stress vs True Strain", x=0.5, y=0.95, xanchor="center",
                                                font=dict(size=20, color="black", family="Arial")),
                                     xaxis=dict(title=dict(text="True Strain (mm/mm)", font=dict(family="Arial", size=16, color="black")),
                                                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                                                gridcolor="lightgrey", gridwidth=1),
                                     yaxis=dict(title=dict(text="True Stress (MPa)", font=dict(family="Arial", size=16, color="black")),
                                                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                                                gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                                     legend=dict(title="Striker Velocity - Wave Method", x=1.0, y=1.0, font=dict(size=12, color="black", family="Arial"),
                                                 bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))

        # Add a text box annotation
        true_stress_strain.add_annotation(
            text=f"Shaper Radius = {radius_labels[shaper_r]}<br>Shaper Thickness = 0.{shaper_t.split('in')[0]} in",
            xref="paper", yref="paper", x=0, y=1, showarrow=False,
            font=dict(size=14, color="black", family="Arial"), align="left",
            bgcolor="white", bordercolor="black", borderwidth=2
        )

        # Loop through available velocity data
        for idx, velocity in enumerate(binout_data.keys()):
            for folder_name, file_path in binout_data[velocity]:
                
                # Create a DataSources object and a Model
                ds = dpf.DataSources(file_path)
                model = dpf.Model(ds)

                # Extract strain data
                total_strain_result = model.results.total_strain()
                strain_fc = total_strain_result.outputs.fields_container()

                strain_incident = strain_fc[0].data  # Incident strain
                strain_transmitted = strain_fc[6].data  # Transmitted strain

                time_freq_support = model.metadata.time_freq_support
                time_values = time_freq_support.time_frequencies.data

                # Assign color for each velocity
                color = mrybm_colors[idx % len(mrybm_colors)]
                try: 
                    true_strain_1, true_strain_2, true_stress_1, true_stress_2 = compute_true_stress_strain(time_values,
                                                                                                        strain_incident,
                                                                                                        strain_transmitted,
                                                                                                        debug=False)
    
                    # Add traces for incident and transmitted strains
                    true_stress_strain.add_trace(go.Scatter(
                        x=-true_strain_1, y=-true_stress_1*1000, mode='lines',
                        name=f"{velocity.split('ms')[0]} m/s - 2-Wave Method", line=dict(color=color, width=2)
                    ))
                    true_stress_strain.add_trace(go.Scatter(
                        x=true_strain_2, y=-true_stress_2*1000, mode='lines',
                        name=f"{velocity.split('ms')[0]} m/s - 1-Wave Method", line=dict(color=color, width=2, dash="dash")
                    ))
                except Exception as e:
                    print(e)

        # Save the interactive plot as an HTML file
        output_filename = f"Velocity_Study_{shaper_t}{shaper_r}.html"
        study_figure_dir = os.path.join(figure_dir, "Velocity_True")
        os.makedirs(study_figure_dir, exist_ok=True)
        true_stress_strain.write_html(os.path.join(study_figure_dir, output_filename))
        print(f"Completed: {shaper_t} thickness, {radius_labels[shaper_r]} radius")


D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_050
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\20ms\shaper_025_050
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\25ms\shaper_025_050
Completed: 025in thickness, 3/16 in radius
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_033
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\20ms\shaper_025_033
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\25ms\shaper_025_033
Completed: 025in thickness, 1/8 in radius
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_025
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\20ms\shaper_025_025
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\25ms\shaper_025_025
Completed: 025in thickness, 3/32 in radius
D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\020in\15ms\shaper_020_050
D:\DynaMat_UTEP\SHPB_P

In [42]:
# Define parameters
shaper_thicknesses = ["025in", "020in", "015in"]  # Now thickness is the variable
striker_velocities = ["15ms", "20ms", "25ms"]  # Keep velocity fixed per plot
shaper_radii = ["_050", "_033", "_025"]  # Keep radius fixed per plot

# Define mapping for legend names
radius_labels = {
    "_050": "3/16 in",
    "_033": "1/8 in",
    "_025": "3/32 in"
}

# Function to find BINOUT files for a specific velocity and radius across all thicknesses
def find_binout_files_for_thickness(base_dir, velocity, radius):
    """
    Find 'binout' files for a given striker velocity and radius across all thicknesses.
    Returns a dictionary where keys are thicknesses, and values are (folder_name, file_path) tuples.
    """
    binout_data = {}
    for thickness in shaper_thicknesses:
        radius_folder = f"shaper_{thickness.split('in')[0]}{radius}"
        thickness_path = os.path.join(base_dir, thickness, velocity, radius_folder)
        print(f"Searching in: {thickness_path}")
        binout_files = []

        # Search for BINOUT files
        for file_path in Path(thickness_path).rglob("*"):
            if file_path.is_file() and file_path.name.lower() == "binout":
                folder_name = file_path.parent.name
                binout_files.append((folder_name, str(file_path)))

        if binout_files:
            binout_data[thickness] = binout_files  # Store per thickness

    return binout_data

# Loop over velocities and shaper radii (keeping them constant per plot)
for striker_v in striker_velocities:
    for shaper_r in shaper_radii:
        
        # Find binout files for the given velocity and radius across all thicknesses
        binout_data = find_binout_files_for_thickness(base_directory, striker_v, shaper_r)

        # Skip if no data is found for this combination
        if not binout_data:
            print(f"No binout data found for {striker_v} velocity, {radius_labels[shaper_r]} radius")
            continue

        true_stress_strain = go.Figure()
        
        true_stress_strain.update_layout(width=1100, height=600, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                                     title=dict(text="True Stress vs True Strain", x=0.5, y=0.95, xanchor="center",
                                                font=dict(size=20, color="black", family="Arial")),
                                     xaxis=dict(title=dict(text="True Strain (mm/mm)", font=dict(family="Arial", size=16, color="black")),
                                                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                                                gridcolor="lightgrey", gridwidth=1),
                                     yaxis=dict(title=dict(text="True Stress (MPa)", font=dict(family="Arial", size=16, color="black")),
                                                tickfont=dict(size=16, color="black", family="Arial"), showgrid=True,
                                                gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                                     legend=dict(title="Shaper Thickness - Wave Method", x=1.0, y=1.0, font=dict(size=12, color="black", family="Arial"),
                                                 bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))

        # Add a text box annotation
        true_stress_strain.add_annotation(
            text=f"Shaper Radius = {radius_labels[shaper_r]}<br>Striker Velocity = {striker_v.split('ms')[0]} m/s",
            xref="paper", yref="paper", x=0, y=1, showarrow=False,
            font=dict(size=14, color="black", family="Arial"), align="left",
            bgcolor="white", bordercolor="black", borderwidth=2
        )

        # Loop through available thickness data
        for idx, thickness in enumerate(binout_data.keys()):
            for folder_name, file_path in binout_data[thickness]:
                
                # Create a DataSources object and a Model
                ds = dpf.DataSources(file_path)
                model = dpf.Model(ds)

                # Extract strain data
                total_strain_result = model.results.total_strain()
                strain_fc = total_strain_result.outputs.fields_container()

                strain_incident = strain_fc[0].data  # Incident strain
                strain_transmitted = strain_fc[6].data  # Transmitted strain

                time_freq_support = model.metadata.time_freq_support
                time_values = time_freq_support.time_frequencies.data

                # Assign color for each thickness
                color = mrybm_colors[idx % len(mrybm_colors)]
                try:
                    true_strain_1, true_strain_2, true_stress_1, true_stress_2 = compute_true_stress_strain(time_values,
                                                                                                            strain_incident,
                                                                                                            strain_transmitted,
                                                                                                            debug=False)
                    # Add traces for incident and transmitted strains
                    true_stress_strain.add_trace(go.Scatter(
                        x=-true_strain_1, y=-true_stress_1*1000, mode='lines',
                        name=f"{thickness} - 2-Wave Method", line=dict(color=color, width=2)
                    ))
                    true_stress_strain.add_trace(go.Scatter(
                        x=true_strain_2, y=-true_stress_2*1000, mode='lines',
                        name=f"{thickness} - 1-Wave Method", line=dict(color=color, width=2, dash="dash")
                    ))
                except Exception as e:
                    print(e)

        # Save the interactive plot as an HTML file
        output_filename = f"Thickness_Study_{striker_v}_{shaper_r}.html"
        study_figure_dir = os.path.join(figure_dir, "Thickness_True")
        os.makedirs(study_figure_dir, exist_ok=True)
        true_stress_strain.write_html(os.path.join(study_figure_dir, output_filename))
        print(f"Completed: {striker_v} velocity, {radius_labels[shaper_r]} radius")


Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_050
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\020in\15ms\shaper_020_050
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\15ms\shaper_015_050
Completed: 15ms velocity, 3/16 in radius
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_033
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\020in\15ms\shaper_020_033
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\15ms\shaper_015_033
Completed: 15ms velocity, 1/8 in radius
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\025in\15ms\shaper_025_025
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\020in\15ms\shaper_020_025
Searching in: D:\DynaMat_UTEP\SHPB_PulseShaper\results\SHPB_pulse_shaper\015in\15ms\shaper_015_025
Completed: 15ms velocity, 3/